# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент Стеклов М. А. БФИ2001
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [125]:
!pip install pyvis
!pip install networkx

In [1]:
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd

from time import time
from queue import Queue
from typing import Hashable, Optional, Callable
from copy import deepcopy
from random import randint, choice, shuffle
from math import inf
from os import listdir
from os.path import isfile, join

In [2]:
def file2graph(filename="lab6_io_files/lab6_input.txt") -> nx.DiGraph:
    res = []
    with open(filename, 'r') as f:
        for line in f:
            print(line, end='')
            res.append(list(map(float, line.strip().split())))
    return nx.from_numpy_matrix(np.array(res), create_using=nx.DiGraph)

In [3]:
def graph_to_file(graph: nx.DiGraph, filename: str):
    matrix: np.ndarray = nx.to_numpy_matrix(graph).tolist()
    with open(filename, 'w') as f:
        for i in range(len(matrix)):
            f.write(' '.join(str(int(el)) for el in matrix[i]) + '\n')


In [4]:
def floyd_warshall(graph: nx.DiGraph, start_node: Hashable,
                   finish_node: Hashable) -> list[Hashable]:
    dist: dict[Hashable, dict[Hashable, float]] = {
        u: {v: inf for v in graph.nodes} for u in graph.nodes}
    next_nodes: dict[Hashable, dict[Hashabel, Optional[Hashable]]] = {
        u: {v: None for v in graph.nodes} for u in graph.nodes}
    for u, v in graph.edges:
        dist[u][v] = graph[u][v]["weight"]
        next_nodes[u][v] = v
    for v in dist:
        dist[v][v] = 0
        next_nodes[v][v] = v
    for k in dist:
        for i in dist:
            for j in dist:
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_nodes[i][j] = next_nodes[i][k]
    if next_nodes[start_node][finish_node] is None:
        return []
    path = [start_node]
    curr_node = start_node
    while curr_node != finish_node:
        curr_node = next_nodes[curr_node][finish_node]
        path.append(curr_node)
    return path
    

In [5]:
def dijkstra_dist_path(graph: nx.DiGraph, start_node: Hashable,
                       finish_node: Hashable) -> tuple[dict[Hashable, float], list[Hashable]]:
    prev: dict[Hashable, Optional[Hashable]] = {node: None for node in graph.nodes}
    dist: dict[Hashable, float] = {node: inf for node in graph.nodes}
    used: dict[Hashable, bool] = {node: False for node in graph.nodes}
    dist[start_node] = 0
    for _ in graph.nodes:
        curr_node = None
        for node in graph.nodes:
            if not used[node] and (curr_node is None or dist[node] < dist[curr_node]):
                curr_node = node
        if dist[curr_node] == inf:
            break
        used[curr_node] = True
        for next_node in graph.succ[curr_node]:
            if ((new_weight := dist[curr_node] + graph[curr_node][next_node]["weight"]) <
                dist[next_node]):
                dist[next_node] = new_weight
                prev[next_node] = curr_node
    path = [finish_node]
    curr_node = finish_node
    while (prev_node := prev[curr_node]) is not None:
        path.append(prev_node)
        curr_node = prev_node
    return dist, list(reversed(path))

def dijkstra_dist(graph: nx.DiGraph, start_node: Hashable,
                  finish_node: Hashable) -> dict[Hashable, float]:
    return dijkstra_dist_path(graph, start_node, finish_node)[0]

def dijkstra(graph: nx.DiGraph, start_node: Hashable,
             finish_node: Hashable) -> list[Hashable]:
    return dijkstra_dist_path(graph, start_node, finish_node)[1]

In [6]:
def bellman_ford_dist_path(graph: nx.DiGraph, start_node: Hashable,
                           finish_node: Optional[Hashable] = None) -> (
    tuple[dict[Hashable, float], list[Hashable]]):
    dist: dict[Hashable, float] = {node: inf for node in graph.nodes}
    dist[start_node] = 0
    prev: dict[Hashable, Optional[Hashable]] = {node: None for node in graph.nodes}
    for _ in range(len(graph.nodes) - 1):
        for u, v in graph.edges:
            if dist[v] > (new_weight := dist[u] + graph[u][v]["weight"]):
                dist[v] = new_weight
                prev[v] = u
    for u, v in graph.edges:
        if dist[v] > dist[u] + graph[u][v]["weight"]:
            assert False, "Negtive cycle detected"
    if finish_node is None:
        return dist, []
    path = [finish_node]
    curr_node = finish_node
    while (prev_node := prev[curr_node]) is not None:
        path.append(prev_node)
        curr_node = prev_node
    return dist, list(reversed(path))

def bellman_ford_dist(graph: nx.DiGraph, start_node: Hashable) -> dict[Hashable, float]:
    return bellman_ford_dist_path(graph, start_node)[0]

def bellman_ford(graph: nx.DiGraph, start_node: Hashable,
                      finish_node: Hashable) -> list[Hashable]:
    return bellman_ford_dist_path(graph, start_node, finish_node)[1]

In [7]:
def johnson(graph: nx.DiGraph, start_node: Hashable, finish_node: Hashable) -> list[Hashable]:
    graph = deepcopy(graph)
    if all(edge_data[2]["weight"] >= 0 for edge_data in graph.edges(data=True)):
        return dijkstra(graph, start_node, finish_node)
    graph.add_node('S')
    for node in graph.nodes:
        if node == 'S':
            continue
        graph.add_edge('S', node, weight=0)
    h = bellman_ford_dist(graph, 'S')
    graph.remove_node('S')
    for edge_data in graph.edges(data=True):
        edge_data[2]['weight'] += h[edge_data[0]] - h[edge_data[1]]
    return dijkstra(graph, start_node, finish_node)

In [8]:
def levit(graph: nx.DiGraph, start_node: Hashable, finish_node: Hashable) -> list[Hashable]:
    m0 = set()
    m1 = set()
    m11 = Queue()
    m12 = Queue()
    m2 = {node for node in graph.nodes if node != start_node}
    dist: dict[Hashable, float] = {node: inf for node in graph.nodes}
    dist[start_node] = 0
    prev: dict[Hashable, Hashable] = dict()
    prev[start_node] = None
    m12.put(start_node)
    m1.add(start_node)
    while len(m1) > 0:
        u = m11.get() if m12.empty() else m12.get()
        m1.remove(u)
        for v in graph.succ[u]:
            if v in m2:
                m11.put(v)
                m1.add(v)
                m2.remove(v)
                if (new_weight := dist[u] + graph[u][v]["weight"]) < dist[v]:
                    dist[v] = new_weight
                    prev[v] = u
            elif v in m1:
                if (new_weight := dist[u] + graph[u][v]["weight"]) < dist[v]:
                    dist[v] = new_weight
                    prev[v] = u
            elif v in m0 and (new_weight := dist[u] + graph[u][v]["weight"]) < dist[v]:
                m12.put(v)
                m1.add(v)
                m0.remove(v)
                dist[v] = new_weight
                prev[v] = u
            m0.add(u)
    path = [finish_node]
    curr_node = finish_node
    while (prev_node := prev[curr_node]) is not None:
        path.append(prev_node)
        curr_node = prev_node
    return list(reversed(path))

In [9]:
def yen(graph: nx.DiGraph, start_node: Hashable,
        finish_node: Hashable, k: int=2) -> list[Hashable]:
    candidates = []
    res = [dijkstra(graph, start_node, finish_node)]
    for _ in range(k - 1):
        for i in range(len(res[-1]) - 1):
            mod_graph = deepcopy(graph)
            root_path = res[-1][:i+1]
            root_path_len = sum(graph[root_path[j]][root_path[j + 1]]["weight"]
                                for j in range(1, i))
            spur_node = root_path[i]
            for res_path in res:
                if root_path == res_path[:i+1] and (
                    (res_path[i], res_path[i+1]) in mod_graph.edges):
                    mod_graph.remove_edge(res_path[i], res_path[i+1])
            for node in root_path:
                if node != spur_node and node in mod_graph.nodes:
                    mod_graph.remove_node(node)
            dist, spur_path = dijkstra_dist_path(mod_graph, spur_node, finish_node)
            if len(spur_path) == 1:
                continue
            spur_path_len = dist[finish_node]
            candidate = root_path + spur_path[1:]
            if candidate not in candidates:
                candidates.append((candidate, root_path_len + spur_path_len))
        if len(candidates) == 0:
            break
        candidates.sort(key=lambda x: x[1], reverse=True)
        res.append(candidates.pop()[0])
    return res
        

In [10]:
def gen_graph(n: int, a: int=10, b: int=50, neg_a: int=-9, neg_b: int=-1, neg_count: int=0):
    g = nx.dorogovtsev_goltsev_mendes_graph(n).to_directed()
    for edge_data in g.edges(data=True):
        edge_data[2]["weight"] = randint(a, b)
    edges_list = list(g.edges())
    for i in range(neg_count):
        u, v = choice(edges_list)
        g[u][v]["weight"] = randint(neg_a, neg_b)
    while True:
        tmp = list(g.edges())[::]
        shuffle(tmp)
        flag = True
        for u, v in tmp:
            if (v, u) in g.edges():
                g.remove_edge(v, u)
                flag = False
                break
        if flag:
            break 
    return g

In [11]:
def prepare_to_show(nt: Network, g: nx.DiGraph, with_weights=True, path=None):
    nt.from_nx(g)
    if with_weights:
        for edge in nt.get_edges():
            edge["value"] = edge["weight"]
    if path is not None:
        path_edges = {(path[i], path[i+1]) for i in range(len(path) - 1)}
        for edge in nt.get_edges():
            if (edge["from"], edge["to"]) in path_edges:
                edge["color"] = "red"

In [12]:
#g = gen_graph(4)
g = file2graph()
for u, v in g.edges:
    g[u][v]["weight"] += 4


0 1 0 0 0 1 6 0 
0 0 -3 0 0 0 0 0 
8 0 0 -1 0 0 0 9 
0 5 0 0 0 7 0 0 
5 0 1 0 0 0 0 0 
0 8 4 0 0 0 0 0 
0 0 4 2 -3 0 0 0 
0 5 0 6 0 0 0 0 


In [13]:
print(nx.dijkstra_path(g, 7, 4))
path = dijkstra(g, 7, 4)
print(path)
print(floyd_warshall(g, 7, 4))
print(bellman_ford(g, 7, 4))
print(levit(g, 7, 4))
print(yen(g, 7, 4, 3))

[7, 1, 2, 0, 6, 4]
[7, 1, 2, 0, 6, 4]
[7, 1, 2, 0, 6, 4]
[7, 1, 2, 0, 6, 4]
[7, 1, 2, 0, 6, 4]
[[7, 1, 2, 0, 6, 4], [7, 3, 1, 2, 0, 6, 4], [7, 3, 5, 2, 0, 6, 4]]


In [14]:
nt = Network(height='500px', width='100%', bgcolor='#222222', font_color='white',
             notebook=True, directed=True)
prepare_to_show(nt, g, path=path)
nt.show('nx.html')

In [19]:
start_node, finish_node = map(int, input("Введите стартовую и конечные вершины через пробел:\n").split())
path = dijkstra(g, start_node, finish_node)

Введите стартовую и конечные вершины через пробел:
2 4


In [20]:
nt = Network(height='500px', width='100%', bgcolor='#222222', font_color='white',
             notebook=True, directed=True)
prepare_to_show(nt, g, path=path)
nt.show('nx.html')

In [15]:
AlgFunc = Callable[[nx.DiGraph, Hashable, Hashable], list[Hashable]]
TestInputs = list[tuple[nx.DiGraph, Hashable, Hashable]]
def test(alg_func: AlgFunc, test_inputs: TestInputs) -> tuple[str, list[float]]:
    res = (alg_func.__name__, [])
    for test_input in test_inputs:
        t = time()
        alg_func(*test_input)
        res[1].append(time() - t)
    return res

def read_test_inputs(path: str="lab6_io_files") -> TestInputs:
    tmp = []
    filenames = [f for f in listdir(path) if isfile(join(path, f))]
    for filename in filenames:
        if filename.startswith("test_graph"):
            _, _, i, s, f = filename[:-4].split("_")
            i, s, f = map(int, (i, s, f))
            graph = file2graph(join(path, filename))
            tmp.append((i, graph, s, f))
    return [(graph, s, f) for _, graph, s, f in sorted(tmp)]
    
test_inputs = read_test_inputs()    

0 0 42 0 0 0 0 0 26 24 0 0 0 0 0 30 0 41 0 33 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 13 39 38 0 27 48 0 22 37 49 0 0 10 42 22 38 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 41 37 29 0 25 0 0 0 32 22 0 13 29 11 47 23 0 0 31 11 0 47 0 47 24 0 0 20 0 0 0 16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
28 0 14 20 0 0 30 0 0 0 15 49 15 0 0 0 0 0 0 0 0 0 0 0 24 12 0 38 0 28 0 0 0 0 0 0 0 0 0 0 0 0 42 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 19 0 0 0 0 0 11 42 19 47 0 0 37 0 25 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 19 0 0 15 0 0 25 36 0 0 0 0 0 0 46 44 32 28 28 0 44 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 27 0 0 0 0 0 0 0 0 0 49 0 0 0 0 0 0 0 0 26 0 13 39 44 17 0 0 0 0 0 0 0 0 0 0 0 0
23 30 0 0 28 49 0 0 0 0 16 0 0 10 18 0 39 0 0 0 0 0 0 22 0 0 0 0 0 0 0 0 0 49 35 39 0 0 0 0 0 0
37 49 0 0 0 0 0 0 0 0 0 48 0 0 0 0 0 41 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 48 0 0 0 0 0 0 0 0 0 0 0 13 0 0 0 0 0 0 0 0 46 0 0
0 0 0 0 0 0 0 0 0 0 0 0 20 0 40 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 16
30 40 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 47 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 22 0 0 0 0 0 0 0 0 0 0 0 33 0 0 0 0 0 0 0 0 0
0 0 0 42 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
24 0 0 0 28 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 46 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 39 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 34 0 0 0 0 0 0 0 0 0 0 0 

In [16]:
test_dict = dict([test(floyd_warshall, test_inputs),
                  test(dijkstra, test_inputs),
                  test(bellman_ford, test_inputs),
                  test(johnson, test_inputs),
                  test(levit, test_inputs),
                  test(yen, test_inputs)])


In [232]:
df = pd.DataFrame(data=test_dict,
                  index=[f"{len(graph.nodes)}, {len(graph.edges)}" for graph, _, _ in test_inputs])
df.index.name = "nodes, edges"
print(df)

              floyd_warshall  dijkstra  bellman_ford   johnson     levit  \
nodes, edges                                                               
3, 3                0.000125  0.000085      0.000055  0.000215  0.000127   
6, 9                0.001087  0.000045      0.000099  0.000187  0.000083   
15, 27              0.002237  0.000114      0.000660  0.000408  0.000185   
42, 81              0.049538  0.000439      0.005754  0.001283  0.000417   
123, 243            0.449530  0.002001      0.044999  0.004259  0.001173   
366, 729           11.872640  0.013884      0.406421  0.022006  0.003588   
1095, 2187        332.093936  0.113965      4.134997  0.135281  0.010626   

                   yen  
nodes, edges            
3, 3          0.000195  
6, 9          0.000479  
15, 27        0.001397  
42, 81        0.006206  
123, 243      0.018748  
366, 729      0.099885  
1095, 2187    1.008538  


In [17]:
graph, start, finish = test_inputs[4]
path = dijkstra(graph, start, finish)
nt = Network(height='500px', width='100%', bgcolor='#222222', font_color='white',
             notebook=True, directed=True)
prepare_to_show(nt, graph, path=path)
nt.show('nx.html')

### Вывод
Я узнал несколько алгоритмов поиска пути в графах